In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
sys.path.insert(0, '/sdadata/zzq/work/HumanProteinAtlasImageClassification')

IMAGE_CHANNELS = 4
IMAGE_SIZE = [512, 512]
MODEL_NAME = 'resnet50'
IMAGES_DIR = '/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/images_grby_landmarks'

# define class

## logger class

In [ ]:
import logging
from logging import handlers

class Logger(object):
    def __init__(self,filename,level='info',\
                 when='D',backCount=3,\
                 fmt='%(asctime)s\n%(message)s'):
        self.level_relations = {
            'debug':logging.DEBUG,
            'info':logging.INFO,
            'warning':logging.WARNING,
            'error':logging.ERROR,
            'crit':logging.CRITICAL
        }
        self.logger = logging.getLogger(filename)
        format_str = logging.Formatter(fmt)#设置日志格式
        self.logger.setLevel(self.level_relations.get(level))#设置日志级别
        sh = logging.StreamHandler()#往屏幕上输出
        sh.setFormatter(format_str) #设置屏幕上显示的格式
        th = handlers.TimedRotatingFileHandler(filename=filename,when=when,backupCount=backCount,\
                                               encoding='utf-8')#往文件里写入#指定间隔时间自动生成文件的处理器
        th.setFormatter(format_str)#设置文件里写入的格式
        self.logger.addHandler(sh) #把对象加到logger里
        self.logger.addHandler(th)

## dataset class

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import tensorflow as tf
# import torch
# from imgaug import augmenters as iaa
# from common.slim.preprocessing.inception_preprocessing import preprocess_image, preprocess_for_train, preprocess_for_eval

# class MyDataset(object):
#     """
#         read_one_sample-->read_one_batch-->preprocess-->getitem
#     """
#     def __init__(self, df, **kw):
#         self.df, self.kw = df, kw
#         self.str_df = self.df2str(df)
#         self.graph = tf.Graph()
#         with self.graph.as_default():
#             self.__one_sample = self.read_one_sample(self.str_df, kw)
#             self.__batch_samples = self.read_batch_samples(self.__one_sample, kw)
#             self.open_session()
#     def __len__(self,):
#         return int(np.ceil(self.df.shape[0]/self.kw['batch_size']))
#     def __getitem__(self, index):
#         res = self.sess.run(self.__batch_samples) #  读取原始样本
#         res[1] = self.preprocess(res[1]) # res[1]为图片 对图像进行预处理
#         return res
#     def open_session(self, gpu_memory_fraction=None):
#         if gpu_memory_fraction is None:
#             config = tf.ConfigProto(log_device_placement=False, allow_soft_placement=True)
#             config.gpu_options.allow_growth = True
#         else:
#             gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
#             config = tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True, allow_soft_placement=True)
#         self.sess = tf.Session(graph=self.graph, config=config)
#         self.sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
#         self.coord = tf.train.Coordinator()
#         self.threads = tf.train.start_queue_runners(sess=self.sess, coord=self.coord)
#     def close_session(self):
#         self.coord.request_stop() 
#         self.coord.join(self.threads)
#         self.sess.close()
#     def df2str(self, df):
#         def other2str(x):
#             return [str(val) for val in x]
#         data = [','.join(other2str(df.iloc[i])) for i in range(df.shape[0])]
#         return data
#     def read_one_sample(self, data, kw):
#         """
#             读取原始样本（未经过预处理）
#         """
#         data = tf.cast(data, tf.string)
#         value = tf.train.slice_input_producer(tensor_list=[data], shuffle=kw['shuffle'],\
#                                               num_epochs=kw['num_epochs'], capacity=self.df.shape[0])[0]
#         #训练集、测试集图片所在目录不同
#         if kw['is_test'] is True:
#             record = tf.decode_csv(value, record_defaults=[['img_name']], field_delim=',') #这里读取的为一行元素
#             img_name = tf.cast(record[0], dtype=tf.string)
#             img_path = tf.string_join([kw['img_dir'], 'test', img_name], separator='/')
#         else:
#             record_defaults =[[0] for i in range(kw['num_labels'])]
#             record_defaults.insert(0, ['img_name'])
#             record = tf.decode_csv(value, record_defaults=record_defaults, field_delim=',') #这里读取的为一行元素
#             img_name = tf.cast(record[0], dtype=tf.string)
#             img_path = tf.string_join([kw['img_dir'], 'train', img_name], separator='/')
#             label = tf.stack(record[1:], axis=-1)
            
#         img = self.read_img(img_path, kw)
        
#         # 训练集测试集返回值不同
#         if kw['is_test'] is True:
#             return img_name, img
#         else:
#             label.set_shape(kw['num_labels'])
#             return  img_name, img, label
#     def read_img(self, img_path, kw):
#         img_red_path = img_path + '_red.png'
#         img_green_path = img_path +'_green.png'
#         img_blue_path = img_path + '_blue.png'
#         img_yellow_path = img_path + '_yellow.png'
        
#         imgR = self.__read_img(img_red_path, kw)
#         imgG = self.__read_img(img_green_path, kw)
#         imgB = self.__read_img(img_blue_path, kw)
#         if IMAGE_CHANNELS == 3:
#             img = tf.concat([imgR, imgG, imgB], axis=-1)
#         else: #为4通道
#             imgY = self.__read_img(img_yellow_path, kw)
#             img = tf.concat([imgR, imgG, imgB, imgY], axis=-1)
#         img.set_shape([512, 512, IMAGE_CHANNELS])
#         return img
#     def __read_img(self, img_path, kw):
#         img = tf.read_file(img_path)
#         img = tf.image.decode_png(img, channels=1)  
#         img.set_shape([512, 512, 1])
#         return img    
#     def preprocess(self, imgs):
        
#         if self.kw['is_test'] is False:
#             if self.kw['img_aug'] is True:
#                 imgs = self.img_augmention(imgs)
        
#         #标准化
#         imgs = imgs/255.0
#         mean, std = [0.08362823, 0.05534997, 0.05827458, 0.08680427], [0.14305845, 0.10747696, 0.15899317, 0.14487827]
#         imgs = (imgs - mean[0:IMAGE_CHANNELS])/std[0:IMAGE_CHANNELS] 
#         imgs = np.transpose(imgs, (0, 3, 1, 2)) #channels first
#         return imgs
#     def img_augmention(self, img):
#         seq = iaa.Sequential([
#             iaa.OneOf([
#                 iaa.Affine(rotate=90),
#                 iaa.Affine(rotate=180),
#                 iaa.Affine(rotate=270),
#                 iaa.Affine(shear=(-16, 16)),
#                 iaa.ElasticTransformation(sigma=(0,3)),
#                 iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
#                 iaa.Fliplr(0.5),
#                 iaa.Flipud(0.5),
                
#             ])], random_order=True)
#         img = seq.augment_images(img)
#         return img
#     def read_batch_samples(self, one_sample, kw):
#         capacity = 1000 + kw['batch_size'] * 3
#         batch_samples = tf.train.batch(tensors=one_sample, \
#                                        batch_size=kw['batch_size'], \
#                                        num_threads=kw['num_threads'], \
#                                        capacity=capacity, allow_smaller_final_batch=True)
#         return batch_samples
#     def visualize(self, ):
#         record = self.__getitem__(0)
#         imgs = record[1]
#         n_rows, n_cols = 4, 4
#         plt.figure(figsize=(n_rows*4, n_cols*4))
#         for i in range(n_rows):
#             for j in range(n_cols):
#                 plt.subplot(n_rows, n_cols, i*n_cols+j+1)
#                 plt.imshow(np.transpose(imgs[i*n_cols+j], [1,2,0])[:,:,:3])
#         plt.tight_layout()
#         plt.show()

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import imgaug as ia
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
class MyDataset(Dataset):
    def __init__(self, df, img_aug=False, is_test=False):
        self.df, self.img_aug, self.is_test = df, img_aug, is_test
        self.img_dir = os.path.join(IMAGES_DIR, 'test' if is_test else 'train')
        self.img_suffixs = ['_green.png', '_red.png', '_blue.png', '_yellow.png']
        # for GRBY
        #self.mean = np.array([0.04531973, 0.05907663, 0.04067749, 0.05924788])
        #self.std = np.array([0.1048129 , 0.12819295, 0.1317363 , 0.12461057])
        # for GRBY_landmarks
        self.mean = np.array([0.06306252, 0.10562667, 0.07949321, 0.10858839])
        self.std = np.array([0.80647908, 0.82780679, 0.82680373, 0.8309615 ])
    def __len__(self,):
        return int(self.df.shape[0])
    def __getitem__(self, index):
        record = self.df.iloc[index].values
        img = self.read_img(record[1], mode=1)
        if self.img_aug is True:
            img = self.img_aug_func(img)
        #数据标准化
        img = img/255.0
        img = (img-self.mean)/self.std
        img = np.transpose(img, (2,0,1))
        img = img[-IMAGE_CHANNELS:]
        if self.is_test is True:
            return record[0], img.astype(np.float32)
        else:
            return record[0], img.astype(np.float32), np.array(record[2:], dtype=np.int32)
    def read_img(self, img_name, mode=0):
        """
            根据图片名称加载四张单色图像
            return: [G, R, B, Y], shape=[512, 512, 4]
        """
        img_path = '/'.join([self.img_dir, img_name])
        if mode==0:
            img_path = [img_path+val for val in self.img_suffixs]
            img = [io.imread(val) for val in img_path]
            img = np.stack(img, axis=-1)
        else:
            img_path = img_path+'.png'
            img = io.imread(img_path)
        return img
    def mark_img(self, img):
        """
            根据R,B,Y三个通道分别标记G通道
            return: G通道加三个被标记的通道，共计四通道
        """
        G, R, B, Y = np.split(img, axis=-1, indices_or_sections=img.shape[-1])# 此时RGBY的shape均为512x512x1
        R, B, Y = R>np.mean(R), B>np.mean(B), Y>np.mean(Y)
        G = np.repeat(G, repeats=3, axis=-1)
        R = ia.SegmentationMapOnImage(R, shape=G.shape).draw_on_image(G)[...,0]
        B = ia.SegmentationMapOnImage(B, shape=G.shape).draw_on_image(G)[...,0]
        Y = ia.SegmentationMapOnImage(Y, shape=G.shape).draw_on_image(G)[...,0]
        img = np.stack([G[...,0], R, B, Y], axis=-1)
        return img
    def img_aug_func(self, img):
        """
            step1: 填充一律用0，因为从对原始图像的可视化可以指导，背景是黑色的
            step2: 注意加噪音，因为部分小的细胞器本来就是一些琐碎的点，这样可以增强模型的鲁棒性
            #注意：seq输入图片的shape为： [N, H, W, C]
        """
        img = np.expand_dims(img, axis=0)
        seq = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=90),
                iaa.Affine(shear=(-16, 16)),#扭曲
                iaa.ElasticTransformation(sigma=(0,3)),#局部变化
                iaa.PiecewiseAffine(scale=(0.01, 0.05), nb_rows=8, nb_cols=8), # 局部扭曲
                iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),#加高斯噪音
            ])], random_order=True)
        img = seq.augment_images(img)[0]
        return img
class MyLoader(object):
    def __init__(self, df, img_aug=False, is_test=False, batch_size=16, shuffle=False, num_workers=1):
        self.dataset = MyDataset(df, img_aug, is_test)
        self.dataloader = DataLoader(self.dataset, batch_size, shuffle, \
                                     num_workers=num_workers, pin_memory=True)
        self.iter = self.__iter__()
        self.batch_size = batch_size
    def __getitem__(self, index):
        """
            为了兼容之前用tensorflow的书写规范
        """
        try:
            tmp = next(self.iter)
        except StopIteration as e:
            self.iter =self.__iter__()
            return self.__getitem__(0)
        return tmp
    def __iter__(self):
        return iter(self.dataloader)
    def __len__(self):
        return int(np.ceil(len(self.dataset)/self.batch_size))
    def visualize(self):
        record = self.__getitem__(0)
        img_names = record[0]
        imgs = record[1]
        n_rows, n_cols = 4, 4
        plt.figure(figsize=(n_rows*4, n_cols*4))
        for i in range(n_rows):
            for j in range(n_cols):
                plt.subplot(n_rows, n_cols, i*n_cols+j+1)
                plt.imshow(np.transpose(imgs[i*n_cols+j], [1,2,0])[:,:,-3:])
                plt.title(s=str(img_names[i*n_cols+j]))
        plt.tight_layout()
        plt.show()

## predictor class

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision.models as models
import importlib
import common.models_pytorch
importlib.reload(common.models_pytorch.resnet)
from common.models_pytorch import resnet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
class MyPredictor(nn.Module):
    def __init__(self, num_output):
        super(MyPredictor, self).__init__()
        if MODEL_NAME is 'resnet50':
            self.features = resnet.resnet50(pretrained=True, in_channels=IMAGE_CHANNELS)
        self.pool = nn.AdaptiveAvgPool2d(output_size=(1,1)) 
        self.classifier = nn.Linear(self.features.out_features, num_output)
    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)  
        x = x.view(x.size(0), -1)
        y = self.classifier(x)
        return y
    def predict(self, x):
        return self.forward(x)


## loss class

In [ ]:
class MyBCELoss(nn.Module):
    def __init__(self):
        super(MyBCELoss, self).__init__()
        self.loss = nn.BCEWithLogitsLoss().to(device)
    def forward(self, target, input):
        loss = self.loss(target=target, input=input)
        return loss

class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.sum(dim=1).mean()
class MyFocalLossWithWeight(nn.Module):
    def __init__(self, alpha, device, gamma=2):
        super(MyFocalLossWithWeight, self).__init__()
        self.alpha = torch.Tensor(alpha).float().to(device)
        self.device = device
        self.gamma = gamma
    def forward(self, input, target):
        max_val = (-input).clamp(min=0)
        logp_ = input + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        logp = logp_ - input
        loss = self.alpha*target*logp + (1-self.alpha)*(1-target)*logp_
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.sum(dim=1).mean()


## work flow

In [ ]:
from sklearn import metrics
from torch.optim.lr_scheduler import LambdaLR
import logging
import tqdm

class WorkFlow(object):
    def __init__(self, classifier=None, loss=None, optim=None, **params):
        self.classifier = classifier
        self.loss = loss
        self.optim = optim
        self.init_params = params
        self.save_path = params['save_path']
    def lr_find(self, train_dataset, n_points=200, n_repeat=4):
        scheduler = LambdaLR(self.optim, lr_lambda=lambda epoch: 1.1**epoch)
        res = {'lr':[], 'loss':[]}
        self.classifier.train()
        i = 0
        min_loss, max_loss = 100000000, -100000000
        while i<n_points:
            tmp_ = []
            for j in range(n_repeat):
                sample_ids, images, labels = train_dataset[0]
                images = torch.FloatTensor(images).to(self.init_params['device'])
                labels = torch.FloatTensor(labels).to(self.init_params['device'])
                outputs = self.classifier(images)
                loss = self.loss(target=labels, input=outputs)
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                tmp_.append(loss.item())
            cur_loss = np.mean(tmp_)
            min_loss = min(min_loss, cur_loss)
            max_loss = max(max_loss, cur_loss)
            res['loss'].append(cur_loss)
            res['lr'].append(scheduler.get_lr())
            if (i>50) and (cur_loss>min_loss) and (cur_loss>((min_loss+max_loss)/2)) :
                break
            i += 1
            print(i, end='\r')
            scheduler.step()
        return res
    def set_lr_weightdecay(self, lr=None, lr_group=None, weight_decay=None, weight_decay_group=None):
        if lr is not None:
            for i, params in enumerate(self.optim.param_groups):
                params['lr'] = lr
        if lr_group is not None:
            for i, params in enumerate(self.optim.param_groups):
                params['lr'] = lr_group[i]
        if weight_decay is not None:
            for i, params in enumerate(self.optim.param_groups):
                params['weight_decay'] = weight_decay
        if weight_decay_group is not None:
            for i, params in enumerate(self.optim.param_groups):
                params['weight_decay'] = weight_decay_group[i]
    def train(self, train_dataset, params, val_dataset=None):
        params['train_steps'] = len(train_dataset) if params['train_steps'] is None else params['train_steps']
        if val_dataset is not None:
            params['valid_steps'] = len(valid_dataset) if params['valid_steps'] is None else params['valid_steps']
        self.train_params = params
        res, s = self.evaluation(val_dataset, params['valid_steps'], pivot=params['pivot'])
        print("start-->", s)
        best_loss = res['loss']
        best_f1 = res['f1']
        if 'history' not in dir(self):
            self.history = {
                'train_loss': [], 'train_pre': [], 'train_rec': [], 'train_f1': [],
                'valid_loss': [], 'valid_pre': [], 'valid_rec': [], 'valid_f1': [],
            }
            self.start_step = 0
        for epoch in range(self.start_step, params['max_epochs']):
            params['lr_scheduler'].step()
            print("learning_rate:", params['lr_scheduler'].get_lr())
            cul_loss = 0.0
            self.classifier.train()
            loss_sum = 0
            pbar = tqdm.tqdm(iterable=range(params['train_steps']), total=params['train_steps'], leave=False)
            for i in pbar:
                # read samples
                sample_ids, images, labels = train_dataset[0]
                images = torch.FloatTensor(images).to(self.init_params['device'])
                labels = torch.FloatTensor(labels.float()).to(self.init_params['device'])
                # Forward pass
                outputs = self.classifier(images)
                loss = self.loss(target=labels, input=outputs)
                loss_sum += loss.item()
                # Backward and optimize
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                s = 'training loss: %.4f' % (loss_sum/(i+1))
                pbar.set_description(s)
            if (epoch+1)% params['display'] == 0:
                ss = "[%04d/%04d]   " %(epoch+1, params['max_epochs'])
                res, s, delta = self.evaluation(train_dataset, params['train_steps'], params['pivot'], train=True)
                
                self.history['train_loss'].append(res['loss'])
                self.history['train_pre'].append(res['precision'])
                self.history['train_rec'].append(res['recall'])
                self.history['train_f1'].append(res['f1'])
                ss += "train%s\t" % s
                if val_dataset is not None:
                    res, s = self.evaluation(val_dataset, params['valid_steps'], params['pivot'])
                    self.history['valid_loss'].append(res['loss'])
                    self.history['valid_pre'].append(res['precision'])
                    self.history['valid_rec'].append(res['recall'])
                    self.history['valid_f1'].append(res['f1'])
                    ss += "valid%s\t" % s
                    if res['loss']<best_loss:
                        self.save(model_name='best_model_loss%.4f'%res['loss'], global_step=epoch+1)
                        best_loss = res['loss']
                    if res['f1']>best_f1:
                        self.save(model_name='best_model_score%.4f'%res['f1'], global_step=epoch+1)
                        best_f1 = res['f1']
                log.logger.info(ss)
                # 调整loss的阈值
                if params['loss_weight_tune'] is True:
                    delta = delta.astype(np.float)/4.0
                    self.loss.alpha = self.loss.alpha + \
                        1*self.loss.alpha*torch.Tensor(delta).float().to(self.init_params['device'])
                    self.loss.alpha = torch.clamp(self.loss.alpha, min=0.05, max=0.95)
                    print(self.loss.alpha)
                #print(ss)
        self.start_step = params['max_epochs']
        return self.history
    def evaluation(self, dataset, batchs=None, pivot=0, train=False):
        """
            如果为训练集，由于shuffle的缘故，num_times=1
            如果为测试集，为了保证估计准确，num_times可大于1
        """
        self.classifier.to(self.init_params['device'])
        y_pred, y_true = [], []
        n_batchs = len(dataset) if batchs is None else batchs
        loss = []
        self.classifier.eval()
        with torch.no_grad():
            pbar = tqdm.tqdm(iterable=range(n_batchs), total=n_batchs, leave=False)
            for i in pbar:
                # read samples
                sample_ids, batch_Xs, batch_ys = dataset[0]
                y_true.append(batch_ys.numpy().astype(np.int32))

                batch_Xs = torch.FloatTensor(batch_Xs).to(self.init_params['device'])
                batch_ys = torch.FloatTensor(batch_ys.float()).to(self.init_params['device'])

                # Forward pass
                batch_y_preds = self.classifier(batch_Xs)
                batch_y_cost = self.loss(batch_y_preds, batch_ys).detach().cpu().numpy()
                batch_y_preds = batch_y_preds.detach().cpu().numpy()

                batch_y_preds = batch_y_preds>pivot #因为这里输出的是logit

                batch_y_preds = batch_y_preds.astype(np.int32)
                loss.append(batch_y_cost)
                y_pred.append(batch_y_preds)
                #print('evaluation [%03d/%03d]' % (i+1, n_batchs), end='\r')
            
        y_true = np.concatenate(y_true, 0)
        y_pred = np.concatenate(y_pred, 0)

        precision = metrics.classification.precision_score(y_true=y_true, y_pred=y_pred, average='macro')
        recall = metrics.classification.recall_score(y_true, y_pred, average='macro')
        f1 = metrics.classification.f1_score(y_true, y_pred, average='macro')
        
        if train is True:
            #为focall loss的权重作准备
            precision_arr = metrics.classification.precision_score(y_true=y_true, y_pred=y_pred, average=None)
            recall_arr = metrics.classification.recall_score(y_true, y_pred, average=None)
            delta = precision_arr+0.1 - recall_arr
        res = {
            'loss': np.mean(loss),
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
        s = "(loss:%.4f, "% res['loss']
        s += "pre:%.4f, "% precision
        s += "rec:%.4f, "% recall
        s += "f1:%.4f) "% f1
        if train is False:
            return res, s
        else:
            return res, s, delta
    def predict(self, dataset, is_test=False, batchs=None):
        self.classifier.to(self.init_params['device'])
        ids, y_pred, y_true = [], [], []
        n_batchs = len(dataset) if batchs is None else batchs
        self.classifier.eval()
        with torch.no_grad():
            pbar = tqdm.tqdm(iterable=range(n_batchs), total=n_batchs, leave=False)
            for i in pbar:
                # read samples
                record = dataset[0]
                sample_ids, batch_Xs = record[0], record[1]
                if is_test is False:
                    batch_ys = record[2].numpy().astype(np.int32)
                    y_true.append(batch_ys)

                ids.append(sample_ids.numpy())
                batch_Xs = torch.FloatTensor(batch_Xs).to(self.init_params['device'])
                # Forward pass
                batch_y_preds = self.classifier.predict(batch_Xs)
                batch_y_preds = batch_y_preds.detach().cpu().numpy()
                y_pred.append(batch_y_preds)
                #print("predict [%04d/%04d]" % (i+1, n_batchs), end='\r')
        ids = np.concatenate(ids, 0).ravel()
        if is_test is False:
            y_true = np.concatenate(y_true, 0)
        y_pred = np.concatenate(y_pred, 0)
        df ={'ids': ids, 'y_true': y_true, 'y_pred':y_pred}
        return df
    def save(self, save_path=None, model_name=None, global_step=1):
        if save_path is None:
            save_path = self.save_path+'/'+self.init_params['model_name']+'/'
        if not os.path.exists(save_path):
            os.mkdir(save_path)
        filepath = save_path+model_name #+'_'+str(global_step)
        torch.save(self.classifier.state_dict(), filepath)
    def restore(self, restore_path=None, model_name=None):
        if restore_path is None:
            restore_path = self.save_path+'/'+self.init_params['model_name']+'/'
        #kpt = tf.train.lastest_checkpoint(restore_path)
        kpt = restore_path + model_name
        self.classifier.load_state_dict(torch.load(kpt))   

## visualizer

In [ ]:
def visualization(history):
    min_ = 100000
    for key in history.keys():
        min_ = min(min_, len(history[key]))
    for key in history.keys():
        history[key] = history[key][:min_]
    df = pd.DataFrame(history)
    rows, cols = 1, 4
    plt.figure(figsize=[4*cols, 4*rows])
    plt.subplot(1,4,1)
    plt.plot(df[['train_loss', 'valid_loss']])
    plt.legend(['train_loss', 'valid_loss'])
    
    plt.subplot(1,4,2)
    plt.plot(df[['train_pre', 'valid_pre']])
    plt.legend(['train_pre', 'valid_pre'])

    plt.subplot(1,4,3)
    plt.plot(df[['train_rec', 'valid_rec']])
    plt.legend(['train_rec', 'valid_rec'])

    plt.subplot(1,4,4)
    plt.plot(df[['train_f1', 'valid_f1']])
    plt.legend(['train_f1', 'valid_f1'])

    plt.tight_layout() #设置默认的间距
    plt.show()

# train_test split

In [ ]:
# # compute statis mean and std
#import pandas as pd
# def compute_mean_std(dataset, is_test=False):
#     res = {'mean':[], 'X2':[]}
#     for i in range(len(dataset)*1):
#         if is_test is False:
#             _, imgs, _ = dataset[0]
#         else:
#             _, imgs = dataset[0]
#         imgs = imgs.numpy()
#         res['mean'].append(np.mean(imgs, axis=(0, 2,3)))
#         res['X2'].append(np.mean(imgs*imgs, axis=(0,2,3)))
#         print('[%03d/%03d]' % (i+1, len(dataset)), end='\r')
#     res['mean'] = np.stack(res['mean'], axis=0)
#     res['mean'] = np.mean(res['mean'], axis=0)
#     res['X2'] = np.stack(res['X2'], axis=0)
#     res['X2'] = np.mean(res['X2'], axis=0)
#     res['std'] = np.sqrt(res['X2']-res['mean']*res['mean'])
#     return res['mean'], res['std']
#GRBY
# test: mean:[0.04531973, 0.05907663, 0.04067749, 0.05924788], std: [0.1048129 , 0.12819295, 0.1317363 , 0.12461057]
# GRBY_landmarks:
#whole: mean: [0.06306252, 0.10562667, 0.07949321, 0.10858839], std: [0.80647908, 0.82780679, 0.82680373, 0.8309615 ]
#train: mean: [0.06974462, 0.07274897, 0.0646682 , 0.0757419 ], std: [1.070733 , 1.027409 , 1.0482745, 1.0303733]
# test: mean:[0.04531973, 0.19292594, 0.11885765, 0.19580479], std:[0.1048129 , 0.2978085 , 0.23873846, 0.3014688 ]

# df_test = '/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/labels/baseline/test.csv'
# df_test = pd.read_csv(df_test)
# df_test.insert(loc=0, column='No', value=df_test.index.values)
# test_dataset = MyLoader(df_test, batch_size=16, num_workers=4, is_test=True)
#compute_mean_std(test_dataset, is_test=True)

# df_train = '/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/labels/baseline/train.csv'
# df_train = pd.read_csv(df_train)
# df_train.insert(loc=0, column='No', value=df_train.index.values)
# train_dataset = MyLoader(df_train, batch_size=16, num_workers=4)
# compute_mean_std(train_dataset, is_test=False)

# a = 31072
# b = 11702
# np.array([1.070733 , 1.027409 , 1.0482745, 1.0303733])*(a/(a+b))+np.array([0.1048129 , 0.2978085 , 0.23873846, 0.3014688 ])*(b/(a+b))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.utils import shuffle
df_train = '/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/labels/baseline/train.csv'
df_train = pd.read_csv(df_train)
df_train.insert(loc=0, column='No', value=df_train.index.values)
train_index, valid_index = train_test_split(df_train.index.values, test_size=0.33, random_state=0, shuffle=True)
train_df, valid_df = df_train.loc[train_index], df_train.loc[valid_index]
train_df = shuffle(train_df)

In [ ]:
train_dataset = MyLoader(train_df, img_aug=True, shuffle=True, batch_size=16, num_workers=4)
valid_dataset = MyLoader(valid_df, batch_size=16, num_workers=4)

In [ ]:
# del train_dataset, valid_dataset

# work_flow

In [ ]:
my_net = MyPredictor(num_output=28).to(device)
my_loss = FocalLoss().to(device)

In [ ]:
my_optimizer = optim.Adam(params=my_net.parameters(), lr=1e-3, weight_decay=1e-4)

In [ ]:
work_flow = WorkFlow(classifier=my_net, loss=my_loss, optim=my_optimizer, \
                     save_path='./result', model_name=MODEL_NAME, device=device)

In [ ]:
log = Logger('./result/'+MODEL_NAME+'/all.log',level='info')

In [ ]:
# #work_flow.set_lr_weightdecay(lr=1e-6)
# #res = work_flow.lr_find(train_dataset, n_points=150, n_repeat=1)
# lr = np.array(res['lr']).reshape((-1, 1))[:,0]
# loss = np.array(res['loss'])
# loss = [np.mean(loss[:i+1]) if i<10 else np.mean(loss[i-5:i+1]) for i, val in enumerate(loss)]
# import seaborn as sns
# plt.figure(figsize=(6*2, 4*1))
# plt.subplot(1,2,1)
# sns.lineplot(x=range(len(lr)),y=lr)
# plt.xlabel('iterations')
# plt.ylabel('learning rate')
# plt.subplot(1,2,2)
# sns.lineplot(x=lr, y=loss)
# plt.semilogx()
# plt.xlabel('learning rate')
# plt.ylabel('loss')
# plt.savefig('./result/'+MODEL_NAME+'/lr_plot.jpeg')
# plt.show()


## fine tune

In [ ]:
# work_flow.set_lr_weightdecay(weight_decay=1e-4)

In [ ]:
#work_flow.set_lr_weightdecay(lr=1e-3)

In [ ]:
# alpha = [0.5863, 0.7342, 0.6891, 0.7998, 0.6907, 0.7061, 0.6084, 0.6644, 0.7492,
#         0.6276, 0.5353, 0.7328, 0.7579, 0.5598, 0.7412, 0.7759, 0.7217, 0.6049,
#         0.7273, 0.6870, 0.7145, 0.6574, 0.7434, 0.7026, 0.7813, 0.6012, 0.6776,
#         0.9500]
# work_flow.loss = MyFocalLossWithWeight(alpha=alpha, device=device)
#work_flow.loss = FocalLoss().to(device)

In [ ]:
#work_flow.restore(model_name='best_model_score0.3439')

## training 

In [ ]:
# tip1: 训练时 平均池化，测试时最大池化; 平均池化可以使网络在训练过程中充分学习，最大池化使得检测的更突出
# tip2: 同样的f1，loss越大越好。loss越大，说明此时没有陷入局部极小值
#work_flow.start_step = lr_sched.last_epoch + 1

In [ ]:
import re
def delete_files(path_dir = './result/'+MODEL_NAME):
    files = sorted(os.listdir(path_dir))
    loss_file = [val for val in files if re.match(pattern=r'[\.\_/a-zA-Z0-9]*loss[0-9\.]*', string=val) is not None]
    score_file = [val for val in files if re.match(pattern=r'[\.\_/a-zA-Z0-9]*score[0-9\.]*', string=val) is not None]
    loss_file = sorted(loss_file)[4:]
    score_file = sorted(score_file)[:-4]
    for file in loss_file:
        tmp = os.path.join(path_dir, file)
        os.remove(tmp)
    for file in score_file:
        tmp = os.path.join(path_dir, file)
        os.remove(tmp)
delete_files()

In [ ]:
from torch.optim import lr_scheduler
lr_sched = lr_scheduler.StepLR(optimizer=work_flow.optim, gamma=0.1, step_size=10, last_epoch=-1)

In [ ]:

train_params = {
    'max_epochs': 50,
    'train_steps': int(len(train_dataset)),
    'valid_steps': int(len(valid_dataset)),
    'display': 1,
    'pivot': 0,
    'loss_weight_tune': False,
    'lr_scheduler':lr_sched,
}
work_flow.train(train_dataset, train_params, valid_dataset)

In [ ]:
visualization(work_flow.history)

In [ ]:
import seaborn as sns
tmp = train_df.drop(labels=['Id'], axis=1).sum(axis=0)/train_df.shape[0]
sns.barplot(x=tmp.index, y=tmp.values, order=tmp.index)

In [ ]:
import seaborn as sns
tmp = valid_df.drop(labels=['Id'], axis=1).sum(axis=0)/valid_df.shape[0]
g = sns.barplot(x=tmp.index, y=tmp.values, order=tmp.index)
for x, y in enumerate(tmp.values):
    g.text(x, y, round(y,4))

# search optimal threshold
    利用leastsq在验证集为每个标签寻找最佳阈值

In [ ]:
import scipy.optimize as opm  #命名为opm， 为了避免与torch.optim 命名重复
from sklearn.metrics import classification
def sigmoid(x):
    return 1.0 / (1.0+np.exp(-x))
def compute_softF1(target, pred, threshold=0.5, scale=50):
    pred = sigmoid(scale*(pred-threshold))
    target = target.astype(np.float)
    score = 2 * (pred * target).sum(axis=0) / ((pred + target).sum(axis=0) + 1e-6)
    return score
def find_optimal_threshold(target, pred):
    num_labels = target.shape[1]
    w0 = 0.5 * np.ones(num_labels) #初始阈值
    alpha = 1e-5  #偏离正常阈值的惩罚系数
    residual_error = lambda w: np.concatenate([compute_softF1(target, pred, w)-1, alpha*(w-0.5)], axis=0)
    w, success = opm.leastsq(residual_error, w0)
    return w
def test_func(targ=None, pred=None):
    if targ is None or pred is None:
        targ = np.random.randint(low=0, high=2, size=(10, 10))
        pred = np.random.rand(10,10)
    #0.5作为预测值
    print("oridinary threshold".center(40, '#'))
    y = (pred>0.5).astype(np.int32)
    tmp = classification.f1_score(y_true=targ, y_pred=y, average='macro')
    print("macro f1: %.4f" % tmp)
    
    # 寻找最佳阈值
    print("optimal threshold".center(40, '#'))
    w = find_optimal_threshold(targ, pred)
    print("optimal threshods:", w)
    y = (pred>w).astype(np.int32)
    tmp = classification.f1_score(y_true=targ, y_pred=y, average='macro')
    print("macro f1: %.4f" % tmp)
    
    print("trunc optimal threshold".center(40, '#'))
    w_trunc = w.copy()
    w_trunc[w_trunc<0.1] = 0.1 #设置最低阈值
    w_trunc[w_trunc>0.9] = 0.9 #设置最大阈值
    print("trunc optimal threshods:", w_trunc)
    y = (pred>w_trunc).astype(np.int32)
    tmp = classification.f1_score(y_true=targ, y_pred=y, average='macro')
    print("trunc macro f1: %.4f" % tmp)
    return w, w_trunc
#test_func()

In [ ]:
# 通过训练集计算最佳阈值
#work_flow.restore(model_name='best_model_score0.6743')
res_train = work_flow.predict(valid_dataset, is_test=False)
res_train['y_prob'] = sigmoid(res_train['y_pred'])
w, w_trunc = test_func(res_train['y_true'], sigmoid(res_train['y_pred']))

In [ ]:
p = classification.precision_score(y_true=res_train['y_true'], y_pred=res_train['y_prob']>0.4, average='macro')
r = classification.recall_score(y_true=res_train['y_true'], y_pred=res_train['y_prob']>0.4, average='macro')
f1 = classification.f1_score(y_true=res_train['y_true'], y_pred=res_train['y_prob']>w_trunc, average='macro')
print(p)
print(r)
print(f1)

In [ ]:
import seaborn as sns
tmp = pd.DataFrame(data=(res_train['y_prob']>w_trunc).astype(np.int32))
# d = tmp.sum(axis=1)
# index = d[d==0].index
# for i in index:
#     k = np.argmax(res_train['y_prob'][i])
#     print(k, end=' ')
#     tmp.iloc[i,k] = 1
#classification.f1_score(y_true=res_train['y_true'], y_pred=tmp.values, average='macro')
tmp = tmp.sum(axis=0)/valid_df.shape[0]
g = sns.barplot(x=tmp.index, y=tmp.values, order=tmp.index)
for x, y in enumerate(tmp.values):
    g.text(x, y, round(y,4))

In [ ]:
# 观察最佳阈值在验证集上的表现
res_valid = work_flow.predict(valid_dataset, is_test=False)
res_valid['y_prob'] = sigmoid(res_valid['y_pred'])
res_valid['y_pred'] = (res_valid['y_prob']>0.5).astype(np.int32)
macro_f1 = classification.f1_score(y_true=res_valid['y_true'], y_pred=res_valid['y_pred'], average='macro')
print("ordianry threshold macro f1 on valid dataset: %.4f" % macro_f1)
res_valid['y_pred'] = (res_valid['y_prob']>w_trunc).astype(np.int32)
macro_f1 = classification.f1_score(y_true=res_valid['y_true'], y_pred=res_valid['y_pred'], average='macro')
print("trunc threshold macro f1 on valid dataset: %.4f" % macro_f1)

# generate result

## generate result

In [ ]:
import pandas as pd
df_test = pd.read_csv('/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/labels/baseline/testA.csv')
test_data_kw = {
    'img_size': IMAGE_SIZE,
    'num_labels':28, 
    'num_epochs':1, #训练集验证集为None,测试集为1
    'img_dir':'/sdadata/zzq/dataset/HumanProteinAtlasImageClassification/images',
    
    'batch_size': 16,
    'shuffle': False, #训练集shuffle为True,否则为False
    'num_threads': 2,  #生成测试结果时用单线程，以保证样本不乱序（多线程由于异步运算会打乱样本的顺序）
    'compute_statis': False,
    'img_aug': False,
    'is_test': True, #区分从哪个目录下导入文件, train or testA
}
#work_flow.restore(model_name='best_model_score0.5249')
test_dataset = MyDataset(df_test, **test_data_kw)
res = work_flow.predict(test_dataset, is_test=True)
del test_dataset

In [ ]:
index = np.argsort(res['ids'])

In [ ]:
del test_dataset

## analyse predicted

In [ ]:
sample_ids = [val.decode() for val in sorted(res['ids'])]
res['y_prob'] = sigmoid(res['y_pred'])
preds = np.array(res['y_prob'][index]>w_trunc-0.02, dtype=np.int32) # w_trunc由 step4:search optimal threshold 得到
df = pd.DataFrame(preds)
df['Id'] = sample_ids
df.set_index(keys=['Id'], drop=True, append=False, inplace=True)
df.head()

In [ ]:
import seaborn as sns
label_count = df.sum(axis=0)/df.shape[0]
g = sns.barplot(x=label_count.index, y=label_count.values, order=label_count.index)
for x, y in enumerate(label_count.values):
    g.text(x, y, round(y,4))
plt.title('label frequence')


In [ ]:
sample_labels_count = df.sum(axis=1)
LCard = sample_labels_count.mean()
sample_labels_count = sample_labels_count.value_counts()/df.shape[0]
sample_labels_count = sample_labels_count.sort_index()
g = sns.barplot(x=sample_labels_count.index, y=sample_labels_count.values, order=sample_labels_count.index)
for x, y in enumerate(sample_labels_count.values):
    g.text(x-0.25, y, round(y,4))
g.title = g.text(1.5, 0.55, 'Label Card: %.2f' % LCard)
plt.show()

## generate submission

In [ ]:
df['Predicted'] = ''

#     temp = np.where(out>= 0)
#     id_list = [ str(x) for x in temp[0] ]
#     pred = ' '.join(id_list)
#     pred_test.append(pred)

def func(row):
    for i, val in enumerate(row.values[:-1]):
        if val == 1:
            row['Predicted'] += (' '+str(i))
    row['Predicted'] = row['Predicted'][1:]  #去掉前面的空格
    return row
df = df.apply(func, axis=1)
df_res = df[['Predicted']]
df_res.to_csv('./result/upload/'+MODEL_NAME +'/score_0.54.csv', index=True, encoding='utf-8')
df_res.head()